In [1]:
from dataset_readers import *
import numpy as np
import pandas as pd

from aif360.metrics import ClassificationMetric
from models import SimpleMLP
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef as mcc

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

In [2]:
datasets = [
    adult_dataset_reader,
    bank_dataset_reader,
    compas_dataset_reader,
    german_dataset_reader
]

In [3]:
def model_initializer(sens_attr):

    model = RandomForestClassifier(n_estimators=1000)

    return model

In [4]:
def eval(dataset_train, dataset_test, unprivileged_groups, privileged_groups, sens_attr):
    model = model_initializer(sens_attr)

    scaler = StandardScaler()
    dataset_train.features = scaler.fit_transform(dataset_train.features)
    dataset_test.features = scaler.transform(dataset_test.features)

    model = model.fit(dataset_train.features, dataset_train.labels.ravel())
    y_pred_prob = model.predict_proba(dataset_test.features)
    pos_ind = np.where(model.classes_ == dataset_test.favorable_label)[0][0]
    y_pred = (y_pred_prob[:, pos_ind] > 0.5).astype(np.float64)

    dataset_pred = dataset_test.copy()
    dataset_pred.labels = y_pred
    metric = ClassificationMetric(
            dataset_test, dataset_pred,
            unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups)

    metrics = dict()
    metrics['overall_acc'] = abs(metric.accuracy())
    metrics['bal_acc'] = abs((metric.true_positive_rate()
                                 + metric.true_negative_rate()) / 2)
    metrics['avg_odds_diff'] = metric.average_abs_odds_difference()
    metrics['disp_imp'] = abs(metric.disparate_impact())
    metrics['stat_par_diff'] = abs(metric.statistical_parity_difference())
    metrics['eq_opp_diff'] = abs(metric.equal_opportunity_difference())
    metrics['theil_ind'] = abs(metric.theil_index())
    metrics['f1_score'] = metric.f1_score()
    metrics.update(metric.performance_measures())
    return metrics

In [5]:
def eval_rcc(dataset_train, dataset_test, sens_attr):
    dataset_train_cp = dataset_train.copy()
    dataset_test_cp = dataset_test.copy()

    sensitive_index_features = dataset_train_cp.feature_names.index(sens_attr)
    sensitive_index_proteced = dataset_train_cp.protected_attribute_names.index(sens_attr)

    # switch protected attributes and labels
    temp = dataset_train_cp.protected_attributes[:,sensitive_index_proteced]
    dataset_train_cp.protected_attributes = dataset_train_cp.labels.reshape(dataset_train_cp.labels.shape[0])
    dataset_train_cp.features[:,sensitive_index_features] = dataset_train_cp.labels.reshape(dataset_train_cp.labels.shape[0])
    dataset_train_cp.labels = temp

    temp = dataset_test_cp.protected_attributes[:,sensitive_index_proteced]
    dataset_test_cp.protected_attributes = dataset_test_cp.labels.reshape(dataset_test_cp.labels.shape[0])
    dataset_test_cp.features[:,sensitive_index_features] = dataset_test_cp.labels.reshape(dataset_test_cp.labels.shape[0])
    dataset_test_cp.labels = temp

    model = model_initializer(sens_attr)

    scaler = StandardScaler()
    dataset_train_cp.features = scaler.fit_transform(dataset_train_cp.features)
    dataset_test_cp.features = scaler.transform(dataset_test_cp.features)

    model = model.fit(dataset_train_cp.features, dataset_train.labels.ravel())

    y_pred_prob = model.predict_proba(dataset_test_cp.features)
    pos_ind = np.where(model.classes_ == dataset_test_cp.favorable_label)[0][0]
    y_pred = (y_pred_prob[:, pos_ind] > 0.5).astype(np.float64)
    y_true = dataset_test_cp.labels.reshape(dataset_test_cp.labels.shape[0])

    rcc = mcc(y_true, y_pred)

    return rcc

In [6]:
def get_n_cov_mean(cov, sensitive_index, cov_count):
    covs = []
    for i in np.argsort(np.abs(cov[sensitive_index]))[::-1]:
        if i != sensitive_index and len(covs) < cov_count:
            covs.append(np.abs(cov[sensitive_index,i]))
    return np.mean(np.array(covs))

In [7]:
dataset_info = []
for dataset_reader in datasets:
    dataset_expanded_train, dataset_train, dataset_val, dataset_test, unprivileged_groups, privileged_groups, sens_attr = dataset_reader()

    num_instances = dataset_expanded_train.features.shape[0] + dataset_test.features.shape[0]
    num_encoded_features = dataset_expanded_train.features.shape[1]
    positive_instances = np.vstack([dataset_expanded_train.labels == dataset_expanded_train.favorable_label, dataset_test.labels == dataset_test.favorable_label]).reshape(num_instances)
    negative_instances = np.vstack([dataset_expanded_train.labels == dataset_expanded_train.unfavorable_label, dataset_test.labels == dataset_test.unfavorable_label]).reshape(num_instances)
    sensitive_index = dataset_expanded_train.protected_attribute_names.index(sens_attr)

    privileged_instances = np.hstack([dataset_expanded_train.protected_attributes[:,sensitive_index] == dataset_expanded_train.privileged_protected_attributes[sensitive_index], dataset_test.protected_attributes[:,sensitive_index] == dataset_test.privileged_protected_attributes[sensitive_index]]).reshape(num_instances)

    unprivileged_instances = np.hstack([dataset_expanded_train.protected_attributes[:,sensitive_index] == dataset_expanded_train.unprivileged_protected_attributes[sensitive_index], dataset_test.protected_attributes[:,sensitive_index] == dataset_test.unprivileged_protected_attributes[sensitive_index]]).reshape(num_instances)

    positive_privileged_instances = np.logical_and(positive_instances, privileged_instances)
    positive_unprivileged_instances = np.logical_and(positive_instances, unprivileged_instances)
    negative_privileged_instances = np.logical_and(negative_instances, privileged_instances)
    negative_unprivileged_instances = np.logical_and(negative_instances, unprivileged_instances)

    metrics = eval(dataset_expanded_train, dataset_test, unprivileged_groups, privileged_groups, sens_attr)

    cov = np.cov(dataset_expanded_train.features.T)
    max_cov = get_n_cov_mean(cov, sensitive_index, 1)
    three_cov_mean = get_n_cov_mean(cov, sensitive_index, 3)

    rcc = eval_rcc(dataset_expanded_train, dataset_test, sens_attr)

    info = {
        'Dataset': dataset_reader.__name__,
        '\# Features' : num_encoded_features,
        '\# Instances': num_instances,
        'Sens. Attr.': sens_attr,
        'Positives': "{:.2%}".format(np.sum(positive_instances)/num_instances),
        'Negatives': "{:.2%}".format(np.sum(negative_instances)/num_instances),
        'Privileged': "{:.2%}".format(np.sum(privileged_instances)/num_instances),
        'Unprivileged': "{:.2%}".format(np.sum(unprivileged_instances)/num_instances),
        'Acc': "{:.3f}".format(metrics['ACC']),
        'MCC': "{:.3f}".format(metrics['MCC']),
        'Stat. Parity.': "{:.3f}".format(metrics['stat_par_diff']),
        'Eq. Opp.': "{:.3f}".format(metrics['eq_opp_diff']),
        'Eq. Odds': "{:.3f}".format(metrics['avg_odds_diff']),
        'Max. Cov.': "{:.3f}".format(max_cov),
        '3-Cov. Mean': "{:.3f}".format(three_cov_mean),
        'RCC': "{:.3f}".format(rcc)

    }
    dataset_info.append(info)

#### Training Dataset shape
(28941, 102)
#### Validation Dataset shape
(7236, 102)
#### Test Dataset shape
(9045, 102)
#### Favorable and unfavorable labels
1.0 0.0
#### Protected attribute names
['sex']
#### Privileged and unprivileged protected attribute values
[array([1.])] [array([0.])]
#### Dataset feature names
['age', 'education-num', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass=Federal-gov', 'workclass=Local-gov', 'workclass=Private', 'workclass=Self-emp-inc', 'workclass=Self-emp-not-inc', 'workclass=State-gov', 'workclass=Without-pay', 'education=10th', 'education=11th', 'education=12th', 'education=1st-4th', 'education=5th-6th', 'education=7th-8th', 'education=9th', 'education=Assoc-acdm', 'education=Assoc-voc', 'education=Bachelors', 'education=Doctorate', 'education=HS-grad', 'education=Masters', 'education=Preschool', 'education=Prof-school', 'education=Some-college', 'marital-status=Divorced', 'marital-status=Married-AF-spouse', 'marital-status=Marr

#### Training Dataset shape
(19512, 57)
#### Validation Dataset shape
(4878, 57)
#### Test Dataset shape
(6098, 57)
#### Favorable and unfavorable labels
1.0 0.0
#### Protected attribute names
['age']
#### Privileged and unprivileged protected attribute values
[array([1.])] [array([0.])]
#### Dataset feature names
['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'job=admin.', 'job=blue-collar', 'job=entrepreneur', 'job=housemaid', 'job=management', 'job=retired', 'job=self-employed', 'job=services', 'job=student', 'job=technician', 'job=unemployed', 'marital=divorced', 'marital=married', 'marital=single', 'education=basic.4y', 'education=basic.6y', 'education=basic.9y', 'education=high.school', 'education=illiterate', 'education=professional.course', 'education=university.degree', 'default=no', 'default=yes', 'housing=no', 'housing=yes', 'loan=no', 'loan=yes', 'contact=cellular', 'contact=telephone', 'mo

#### Training Dataset shape
(3946, 401)
#### Validation Dataset shape
(987, 401)
#### Test Dataset shape
(1234, 401)
#### Favorable and unfavorable labels
0.0 1.0
#### Protected attribute names
['sex', 'race']
#### Privileged and unprivileged protected attribute values
[array([1.]), array([1.])] [array([0.]), array([0.])]
#### Dataset feature names
['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'age_cat=25 - 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'c_charge_degree=F', 'c_charge_degree=M', 'c_charge_desc=Abuse Without Great Harm', 'c_charge_desc=Agg Abuse Elderlly/Disabled Adult', 'c_charge_desc=Agg Assault W/int Com Fel Dome', 'c_charge_desc=Agg Battery Grt/Bod/Harm', 'c_charge_desc=Agg Fleeing and Eluding', 'c_charge_desc=Agg Fleeing/Eluding High Speed', 'c_charge_desc=Aggr Child Abuse-Torture,Punish', 'c_charge_desc=Aggrav Battery w/Deadly Weapon', 'c_charge_desc=Aggrav Child Abuse-Agg Battery', 'c_charge_desc=Aggrav Child

In [11]:
dataset_info_df = pd.DataFrame(dataset_info).set_index(['Dataset'])
dataset_info_df

,\# Features,\# Instances,Sens. Attr.,Positives,Negatives,Privileged,Unprivileged,Acc,MCC,Stat. Parity.,Eq. Opp.,Eq. Odds,Max. Cov.,3-Cov. Mean,RCC
Dataset,,,,,,,,,,,,,,,
adult_dataset_reader,102,45222,sex,24.78%,75.22%,67.50%,32.50%,0.842,0.563,0.193,0.099,0.098,0.527,0.418,0.200
bank_dataset_reader,57,30488,age,12.66%,87.34%,97.17%,2.83%,0.903,0.516,0.115,0.038,0.053,0.332,0.232,-0.064
compas_dataset_reader,401,6167,race,54.45%,45.55%,34.05%,65.95%,0.374,-0.248,0.129,0.044,0.115,0.827,0.532,0.091
german_dataset_reader,58,1000,sex,70.00%,30.00%,69.00%,31.00%,0.620,0.000,0.034,0.012,0.025,0.626,0.354,0.114


,\# Features,\# Instances,Sens. Attr.,Positives,Negatives,Privileged,Unprivileged,Acc,MCC,Stat. Parity.,Eq. Opp.,Eq. Odds,Max. Cov.,3-Cov. Mean,RCC
Dataset,,,,,,,,,,,,,,,
adult_dataset_reader,102,45222,sex,24.78%,75.22%,67.50%,32.50%,0.842,0.563,0.193,0.099,0.098,0.527,0.418,0.200
bank_dataset_reader,57,30488,age,12.66%,87.34%,97.17%,2.83%,0.903,0.516,0.115,0.038,0.053,0.332,0.232,-0.064
compas_dataset_reader,401,6167,race,54.45%,45.55%,34.05%,65.95%,0.374,-0.248,0.129,0.044,0.115,0.827,0.532,0.091
german_dataset_reader,58,1000,sex,70.00%,30.00%,69.00%,31.00%,0.620,0.000,0.034,0.012,0.025,0.626,0.354,0.114


In [12]:
dataset_info_df_transpose = dataset_info_df.transpose()
dataset_info_df_transpose

Dataset,adult_dataset_reader,bank_dataset_reader,compas_dataset_reader,german_dataset_reader
\# Features,102,57,401,58
\# Instances,45222,30488,6167,1000
Sens. Attr.,sex,age,race,sex
Positives,24.78%,12.66%,54.45%,70.00%
Negatives,75.22%,87.34%,45.55%,30.00%
Privileged,67.50%,97.17%,34.05%,69.00%
Unprivileged,32.50%,2.83%,65.95%,31.00%
Acc,0.842,0.903,0.374,0.620
MCC,0.563,0.516,-0.248,0.000
Stat. Parity.,0.193,0.115,0.129,0.034


Dataset,adult_dataset_reader,bank_dataset_reader,compas_dataset_reader,german_dataset_reader
\# Features,102,57,401,58
\# Instances,45222,30488,6167,1000
Sens. Attr.,sex,age,race,sex
Positives,24.78%,12.66%,54.45%,70.00%
Negatives,75.22%,87.34%,45.55%,30.00%
Privileged,67.50%,97.17%,34.05%,69.00%
Unprivileged,32.50%,2.83%,65.95%,31.00%
Acc,0.842,0.903,0.374,0.620
MCC,0.563,0.516,-0.248,0.000
Stat. Parity.,0.193,0.115,0.129,0.034


In [10]:
dataset_info_df_transpose.to_latex(('dataset_info.tex'))